In [ ]:
import numpy as np
import gdspy
import matplotlib.pyplot as plt

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2
import snakenmake
from snakenmake import *

In [ ]:
def plot_cell(cell, exclude=(2,)):
    # FROM: https://github.com/heitzmann/gdspy/issues/42
    poly_dict = cell.get_polygons(by_spec=True)
    plt.figure(figsize=(20, 12))
    for layer_datatype, polys in poly_dict.items():
        if layer_datatype[0] in exclude:
            continue
        for poly in polys:
            plt.fill(*poly.T, lw=0.5, ec="r", fc="none")
    plt.axes().set_aspect("equal", "datalim")


def write_gds(main_cell, filename, unit=1.0e-6, precision=1.0e-9):
    writer = gdspy.GdsWriter(filename, unit=unit, precision=precision)
    for cell in [main_cell] + list(main_cell.get_dependencies(True)):
        writer.write_cell(cell)
    writer.close()

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
chip_dims = np.array([23e3, 13e3])

In [ ]:
fov = 6.5 * 2048 / 40
fov

In [ ]:
23e3 * 13e3 / fov**2

# Text testing

In [ ]:
import text

In [ ]:
text.DEFAULT_FACE.max_advance_height

In [ ]:
text.DEFAULT_FACE.size.height

In [ ]:
text.DEFAULT_FACE.descender

In [ ]:
(
    text.DEFAULT_FACE.ascender - text.DEFAULT_FACE.descender
)  # + text.DEFAULT_FACE.max_advance_height

In [ ]:
# main_cell = wafer([chip('Big Snake S3 FC90 L35 W1.5 LG20 TS2', snake, split=3, draw_trenches=True)]*3, "Big snakes\n180503")
# main_cell = chip('Big Snake S3 FC90 L35 W1.5 LG20 TS2', snake, split=3, draw_trenches=False)
# main_cell = Text2('testing', 600, position=(0,0))
def character(char, size, face=text.DEFAULT_FACE, points_per_segment=10):
    cell = Cell("~{}".format(char))
    polygons, metadata = text.render_character(
        char, size, face=face, points_per_segment=points_per_segment
    )
    cell.add(g.PolygonSet(polygons))
    return cell, metadata


def Text2(
    text,
    size,
    position=(0, 0),
    horizontal=True,
    angle=0,
    x_reflection=False,
    layer=0,
    datatype=0,
    face=text.DEFAULT_FACE,
    points_per_segment=10,
):
    if not horizontal or angle != 0:
        raise NotImplementedError
    position = np.array(position)
    offset = np.zeros(2)
    face.set_char_size(size * 64)
    linespace = face.size.height / 64
    cells = []
    for char in text:
        if char == "\n":
            offset[0] = 0
            offset[1] -= linespace
            continue
        elif char == " ":
            face.load_char(char)
            offset[0] += face.glyph.metrics.horiAdvance
            continue
        char_cell, metadata = character(
            char, size, face=face, points_per_segment=points_per_segment
        )
        cells.append(g.CellReference(char_cell, position + offset))
        offset[0] += metadata["horiAdvance"]
    return cells


# main_cell, md = character_cell('s', 100*64)
main_cell = Cell("main")
cells = Text2("this is a\ntest  foo u", 600, position=(0, 0))
main_cell.add(cells)
print(md)
plot_cell(main_cell)

# Old

In [ ]:
from snakenmake import *

main_cell = wafer(
    [chip("Big Snake S3 FC90 L35 W1.5 LG20 TS2", snake, split=3, draw_trenches=True)]
    * 3,
    "Big snakes\n180503",
)
write_gds(main_cell, "bigsnakes2.gds")

In [ ]:
from snakenmake import *

metadata = {}
main_cell = wafer(
    [
        chip(
            "Big Snake S3 FC{fc} L35 W1.5 TS{ts} LG20".format(fc=fc, ts=ts),
            snake,
            split=3,
            feeding_channel_width=fc,
            trench_spacing=ts,
            draw_trenches=True,
            metadata=metadata,
        )
        for fc in (90, 65)
        for ts in (1, 1.5, 2)
    ],
    "Big Snakes \n BS/JQS 180508",
)
write_gds(main_cell, "bigsnakes2.gds")

In [ ]:
metadata

In [ ]:
from snakenmake import *

metadata = {}
main_cell = wafer(
    [
        chip(
            "Big Snake S3 FC{fc} L35 W1.5 TS{ts} LG20".format(fc=fc, ts=ts),
            snake,
            split=3,
            feeding_channel_width=fc,
            trench_spacing=ts,
            draw_trenches=True,
            tick_labels=False,
            metadata=metadata,
        )
        for fc in (90,)
        for ts in (1, 1.5, 2)
    ],
    "Big Snakes Backup \n BS/JQS 180508",
)
write_gds(main_cell, "bigsnakes2backup.gds")